In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.zillow.com/king-county-wa/?searchQueryState={%22pagination%22:{},%22usersSearchTerm%22:%22King%20County,%20WA%22,%22mapBounds%22:{%22west%22:-122.67175587695311,%22east%22:-120.92218312304686,%22south%22:46.89011178033753,%22north%22:47.971419225684436},%22regionSelection%22:[{%22regionId%22:207,%22regionType%22:4}],%22mapZoom%22:9,%22filterState%22:{%22sortSelection%22:{%22value%22:%22globalrelevanceex%22}},%22isListVisible%22:true,%22isMapVisible%22:true}'
res = requests.get(url)

In [3]:
print(res.raise_for_status())
soup = BeautifulSoup(res.text)
type(soup)

None


bs4.BeautifulSoup

In [4]:
print(soup.prettify())

<html>
 <head>
  <meta content="noindex, nofollow" name="robots"/>
  <link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/>
  <script>
   window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
  </script>
  <script src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0" type="text/javascript">
  </script>
  <script>
   function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        

In [5]:
from selenium import webdriver
import sys
from selenium.webdriver.chrome.options import Options 

In [6]:
options = Options()
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
# options.add_argument('--headless')
options.add_argument('--window-size=500,300')

In [11]:
chrome_path = r'\Users\User\Documents\chromedriver'

In [12]:
driver = webdriver.Chrome(chrome_path, options=options)

In [13]:
driver.set_window_size(1400,1000)

In [14]:
driver.get(url)

In [15]:
driver.find_element_by_class_name('list-card-heading')

<selenium.webdriver.remote.webelement.WebElement (session="945ae2a8da0536bea44330c4bbf0b24e", element="581cbcf9-7bb9-40dd-8601-4109cad55272")>

In [16]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)

In [35]:
#price
float(soup.findAll('div', {'class' : 'list-card-price'})[0].contents[0].replace('$', '').replace(',', ''))

460000.0

In [18]:
#beds
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[0].contents[0])

3.0

In [19]:
#baths
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[1].contents[0])

2.0

In [20]:
#sqft
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[2].contents[0].replace(',', ''))

1710.0

In [47]:
# there are 40 listings per page, thats why the range is 40
# some listings may have missing information, that's why there are exceptions
x = 0
bedrooms = []
bathrooms = []
sqft = []
price = []
p = list(range(40))
for v in p:
    if len(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents) == 3:
        try:
            price.append(float(soup.findAll('div', {'class' : 'list-card-price'})[x].contents[0].replace('$', '').replace(',', '')))
            bedrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[0].contents[0]))
            bathrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[1].contents[0]))
            sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
            x = x + 1
        except:
            if len(price) > len(bedrooms):
                bedrooms.append(None)
                bathrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[1].contents[0]))
                sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
                x = x + 1
            elif len(bedrooms) > len(bathrooms):
                bathrooms.append(None)
                sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
                x = x + 1
            else:
                sqft.append(None)
                x = x + 1
            continue
    else:
        x = x + 1
        

In [74]:
# each house listing has a script that contained extra 
# information like the url, zipcode, latitude and longitude
houses = soup.findAll('script', {'type' : 'application/ld+json'})

In [75]:
ex = houses[0].text
ex

'{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"42617 Se 169th St, North Bend, WA 98045","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,710"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"42617 SE 169th St ","addressLocality":"North Bend","addressRegion":"WA","postalCode":"98045"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":47.4475,"longitude":-121.772199},"url":"https://www.zillow.com/homedetails/42617-SE-169th-St-North-Bend-WA-98045/49132596_zpid/"}'

In [76]:
import ast

_dict = ast.literal_eval(ex)
_dict

{'@type': 'SingleFamilyResidence',
 '@context': 'http://schema.org',
 'name': '42617 Se 169th St, North Bend, WA 98045',
 'floorSize': {'@type': 'QuantitativeValue',
  '@context': 'http://schema.org',
  'value': '1,710'},
 'address': {'@type': 'PostalAddress',
  '@context': 'http://schema.org',
  'streetAddress': '42617 SE 169th St ',
  'addressLocality': 'North Bend',
  'addressRegion': 'WA',
  'postalCode': '98045'},
 'geo': {'@type': 'GeoCoordinates',
  '@context': 'http://schema.org',
  'latitude': 47.4475,
  'longitude': -121.772199},
 'url': 'https://www.zillow.com/homedetails/42617-SE-169th-St-North-Bend-WA-98045/49132596_zpid/'}

In [83]:
_dict['address']['postalCode']

'98045'

In [84]:
_dict['geo']['latitude']

47.4475

In [85]:
_dict['url']

'https://www.zillow.com/homedetails/42617-SE-169th-St-North-Bend-WA-98045/49132596_zpid/'

In [93]:
# Some houses had two scripts, that's why there is an execption
# so we just skip the extra script and go to the next house
t = 0
p = list(range(52))
zipcode = []
lat = []
long = []
url = []
for v in p:
    try:
        ex = houses[t].text
        _dict = ast.literal_eval(ex)
        zipcode.append(_dict['address']['postalCode'])
        lat.append(_dict['geo']['latitude'])
        long.append(_dict['geo']['longitude'])
        url.append(_dict['url'])
        t = t + 1
    except:
        t = t + 1
        continue

In [94]:
len(lat)

39

In [97]:
import pandas as pd

In [320]:
zillowdf = pd.DataFrame()
zillowdf['url'] = url
zillowdf['lat'] = lat
zillowdf['long'] = long
zillowdf['zipcode'] = zipcode
zillowdf['bedrooms'] = bedrooms
zillowdf['bathrooms'] = bathrooms
zillowdf['sqft'] = sqft
zillowdf['price'] = price

In [323]:
zillowdf

,url,lat,long,zipcode,bedrooms,bathrooms,sqft,price
0,/homedetails/4911-NE-Laurelcrest-Ln-Seattle-WA...,47.653084,-122.275773,98105,7.0,9.00,9590.0,13750000.0
1,/homedetails/6602-123rd-Ave-NE-Kirkland-WA-980...,47.664966,-122.175160,98033,4.0,2.00,1686.0,925000.0
2,/homedetails/7351-19th-Ave-NE-Seattle-WA-98115...,47.682788,-122.308074,98115,4.0,2.00,1660.0,799900.0
3,/homedetails/352-N-80th-St-Seattle-WA-98103/48...,47.687164,-122.352949,98103,2.0,2.00,1050.0,545000.0
4,/homedetails/308-E-Republican-St-APT-512-Seatt...,47.623230,-122.326870,98102,1.0,1.00,732.0,345000.0
5,/homedetails/23232-26th-Ave-S-Des-Moines-WA-98...,47.392834,-122.299826,98198,4.0,2.00,1910.0,349950.0
6,/homedetails/2040-Westlake-Ave-N-Seattle-WA-98...,47.637384,-122.339657,98109,2.0,1.00,923.0,750000.0
7,/homedetails/11714-SE-65th-St-Bellevue-WA-9800...,47.545199,-122.183726,98006,3.0,NaN,2000.0,798800.0
8,/homedetails/2903-124th-Ave-NE-Bellevue-WA-980...,47.635945,-122.173301,98005,4.0,3.00,2230.0,1559000.0
9,/homedetails/3203-S-Lane-St-Seattle-WA-98144/4...,47.596424,-122.291102,98144,4.0,4.00,3200.0,1750000.0


In [324]:
zillowdf.to_csv(path_or_buf='~/Documents/ws_Zillow_KC_listings_CL.csv')

In [98]:
driver.quit()